In [64]:
from pathlib import Path
import re
import time
from sqlalchemy import inspect as sqlalchemy_inspect
from litellm import completion
from dotenv import load_dotenv

# -------------------------------
# Environment setup
# -------------------------------
load_dotenv()  # Loads all API keys from .env
MODEL_PRICES = {
    "gpt-4o-mini": {"input": 0.00000015, "output": 0.00000060},
    "gpt-5": {"input": 0.00000125, "output": 0.00001000},
    "gpt-5-mini": {"input": 0.00000025, "output": 0.00000200},
    "gpt-4-turbo": {"input": 0.000010, "output": 0.000030},  
    "gpt-4.1": {"input": 0.000005, "output": 0.000020},  # speculative
    "claude-3-opus": {"input": 0.000015, "output": 0.000075},
    "claude-sonnet-4-5": {"input": 0.000003, "output": 0.000015},  # ✅ new
    "deepseek-coder": {"input": 0.00000010, "output": 0.00000010},
    "mistral-large": {"input": 0.000004, "output": 0.000012},
}

# Global counters
TOTAL_TOKENS = 0
TOTAL_COST = 0.0
TOTAL_START = time.time()

def estimate_cost(model: str, prompt_tokens: int, completion_tokens: int) -> float:
    """
    Estimate total USD cost of an LLM call.

    Units:
      - prompt_tokens: number of input tokens sent to the model
      - completion_tokens: number of output tokens returned by the model
      - rates: USD cost per token (input/output) for the given model
      - returns: total estimated cost in USD (float)
    """
    rates = MODEL_PRICES.get(model, {"input": 0.000003, "output": 0.000003})

    input_cost = prompt_tokens * rates["input"]
    output_cost = completion_tokens * rates["output"]
    total_cost = input_cost + output_cost

    # Round to 6 decimal places for readability and consistency
    return round(total_cost, 6)
# -------------------------------
# Utility functions
# -------------------------------
def clean_code(text: str) -> str:
    """Remove markdown-style fences like ```python or ```html."""
    return re.sub(r"```[a-zA-Z]*\n?|```", "", text).strip()

def ask_model(model: str, prompt: str, temperature: float = 0.4):
    """
    Unified interface for all LLM providers via LiteLLM.
    Works with GPT, Claude, DeepSeek, Mistral, Ollama, etc.
    Returns (response_text, latency_seconds, usage_dict)
    """

    global TOTAL_TOKENS, TOTAL_COST

    start = time.time()

    #Build the base kwargs
    kwargs = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are an expert FastAPI full-stack developer. Return only raw code, no markdown."},
            {"role": "user", "content": prompt},
        ],
    }

    if not any(m["role"] == "user" for m in kwargs["messages"]):
        kwargs["messages"].append({"role": "user", "content": prompt})
        
    # Only include temperature if NOT GPT-5
    if not model.lower().startswith("gpt-5"):
        kwargs["temperature"] = temperature

    try:
        response = completion(**kwargs)
    except TypeError:
        # fallback if a model (like gpt-5) rejects 'temperature'
        if "temperature" in kwargs:
            del kwargs["temperature"]
        response = completion(**kwargs)

    end = time.time()
    latency = end - start

    # Extract usage + cost
    content = response["choices"][0]["message"]["content"].strip()
    usage = response.get("usage", {})
    prompt_tokens = usage.get("prompt_tokens", 0)
    completion_tokens = usage.get("completion_tokens", 0)
    total_tokens = prompt_tokens + completion_tokens
    cost = estimate_cost(model, prompt_tokens, completion_tokens)
    usage["cost_usd"] = cost
    usage["cost_usd"] = cost

    # Accumulate global totals
    TOTAL_TOKENS += total_tokens
    TOTAL_COST += cost

    print(f"{latency:.2f}s | {total_tokens} tokens | ${cost:.5f}")
    return clean_code(content), latency, usage



# -------------------------------
# Configurable model selector
# -------------------------------
DEFAULT_MODEL = "gpt-4o-mini"  # You can change this to claude-3-opus, deepseek-coder, etc.

def ask(prompt: str, model: str = DEFAULT_MODEL):
    """Generic wrapper for app-building tasks."""
    #print(f"\nUsing model: {model}")
    response, latency, usage = ask_model(model, prompt)
    #print(f"Took {latency:.2f}s | Tokens: {usage.get('total_tokens', 'n/a')}")
    return response


# -------------------------------
# Step 1: Analyze app idea
# -------------------------------
def analyze_prompt(user_prompt: str, model: str = DEFAULT_MODEL) -> str:
    print(f"Analyzing app idea: {user_prompt}")
    analysis_prompt = f"""
    The user wants a database-backed app.
    Describe the entities, fields, and relationships.

    User description:
    {user_prompt}
    """
    analysis = ask(analysis_prompt, model)
    print("Schema plan:\n", analysis)
    return analysis


# -------------------------------
# Step 2: Generate schema.py
# -------------------------------
def generate_schema(analysis: str, model: str = DEFAULT_MODEL):
    print("Generating SQLAlchemy schema...")
    prompt = f"""
    Write valid Python 3.11 code defining SQLAlchemy 2.0 ORM models using DeclarativeBase.

    Requirements:
    - Define class Base(DeclarativeBase)
    - Each ORM class must subclass Base (e.g., `class Customer(Base):`) — do NOT use `Base()`
    - Import from sqlalchemy.orm: DeclarativeBase, Mapped, mapped_column, relationship
    - Import from sqlalchemy: Integer, String, Text, DateTime, ForeignKey, Column, Table
    - Include __allow_unmapped__ = True in each class
    - Each model must define a unique __tablename__
    - Use mapped_column instead of Column *only inside ORM classes*
    - When defining association tables with Table(), use Column() not mapped_column()
    - When a table has multiple foreign keys to the same parent, specify `foreign_keys` explicitly in relationship()
    - Avoid using reserved names like 'metadata', 'query', or 'registry'
    - Use 'meta_data' instead of 'metadata'
    - No markdown or explanations
    - Only create models that are explicitly described in the user input
    - Do NOT invent entities not mentioned in the description.
    - If unsure, leave it out.
    
    Schema description:
    {analysis}
    """
    code = ask(prompt, model)
    code = code = code.replace('(Base())', '(Base)')
    Path("generated/schema.py").write_text(code)
    print("Saved generated/schema.py")


# -------------------------------
# Step 3: Extract model names
# -------------------------------
def extract_model_names():
    code = Path("generated/schema.py").read_text()
    return re.findall(r"class\s+(\w+)\s*\(Base\)", code)


# -------------------------------
# Step 4: Get model fields dynamically
# -------------------------------
def get_model_fields(model_name):
    """Return dict of model fields for dynamic Form() generation."""
    from generated import schema
    model = getattr(schema, model_name)
    mapper = sqlalchemy_inspect(model)
    return {col.key: col.type.python_type.__name__ for col in mapper.columns if col.key != "id"}


# -------------------------------
# Step 5: Generate backend API
# -------------------------------
def generate_api():
    """
    Generates a FastAPI backend (generated/api.py) with form-compatible endpoints.
    Use valid Python 3.11 code.
    This ensures HTMX forms (which send Form data) always match backend expectations.
    """
    print("Generating FastAPI backend...")

    from generated import schema
    import inspect, re
    from pathlib import Path

    # Dynamically extract model classes from schema
    models = [
        name for name, obj in inspect.getmembers(schema)
        if inspect.isclass(obj) and hasattr(obj, "__tablename__")
    ]

    # Build safe import line
    imports = ", ".join(["Base"] + models)

    api_code = f"""from fastapi import FastAPI, HTTPException, Form, Depends
from fastapi.middleware.cors import CORSMiddleware
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine

try:
    from .schema import {imports}
except ImportError:
    from .schema import Base

SQLALCHEMY_DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={{"check_same_thread": False}})
SessionLocal = sessionmaker(bind=engine, autocommit=False, autoflush=False)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

app = FastAPI(title="Generated API", version="1.0")

# CORS for frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://127.0.0.1:8001", "http://localhost:8001", "http://127.0.0.1", "http://localhost"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

Base.metadata.create_all(bind=engine)
"""

    # helper: convert CamelCase → snake_case
    def snake_case(name):
        return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()

    # generate CRUD routes
    for model_name in models:
        endpoint = snake_case(model_name)
        api_code += f"""

@app.post("/{endpoint}/")
async def create_{endpoint}(
"""
        # dynamically introspect model columns
        model_cls = getattr(schema, model_name)
        params = []
        assignments = []
        for col_name, col_type in model_cls.__annotations__.items():
            if col_name == "id":
                continue
            # default all to Form(...) for HTMX compatibility
            params.append(f"    {col_name}: str = Form(...)")
            assignments.append(f"{col_name}={col_name}")

        api_code += ",\n".join(params) + ",\n    db: Session = Depends(get_db)\n):\n"
        api_code += f"    new_item = {model_name}({', '.join(assignments)})\n"
        api_code += "    db.add(new_item)\n    db.commit()\n    db.refresh(new_item)\n    return new_item\n"

        # READ
        api_code += f"""
@app.get("/{endpoint}/")
async def read_{endpoint}s(db: Session = Depends(get_db)):
    return db.query({model_name}).all()
"""

        # DELETE
        api_code += f"""
@app.delete("/{endpoint}/{{item_id}}")
async def delete_{endpoint}(item_id: int, db: Session = Depends(get_db)):
    item = db.query({model_name}).filter({model_name}.id == item_id).first()
    if not item:
        raise HTTPException(status_code=404, detail="{model_name} not found")
    db.delete(item)
    db.commit()
    return {{"message": "{model_name} deleted"}}
"""

    Path("generated/api.py").write_text(api_code)
    print("API generated successfully with HTMX-compatible Form endpoints!")


# -------------------------------
# Step 6: Generate frontend.py
# -------------------------------
def generate_frontend(models: list, model: str = DEFAULT_MODEL):
    print("Generating frontend app...")
    model_imports = ", ".join(models)
    prompt = f"""
Write valid Python 3.11 code for a FastAPI app named 'app' that:
- Imports Base, {model_imports} from .schema
- Imports get_db from .api
- Uses Jinja2Templates("./generated/templates")
- Renders index.html on '/'
- Includes CRUD endpoints using Form() params and RedirectResponse
- Does NOT use response_model or ORM types in function annotations
- Returns RedirectResponse after POST and {{}} dicts for delete
- All routes must include response_model=None in their decorators
- No markdown fences
"""
    code = ask(prompt, model)
    Path("generated/frontend.py").write_text(code)
    print("Saved generated/frontend.py")


# -------------------------------
# Step 7: Generate index.html
# -------------------------------
# def generate_index_html(models: list, model: str = DEFAULT_MODEL):
#     print("Generating index.html...")
#     Path("generated/templates").mkdir(parents=True, exist_ok=True)
#     prompt = f"""
# Write a simple responsive Jinja2 HTML page using TailwindCSS and HTMX
# for models {models}. It should:
# - Display data tables and forms for each model
# - Use hx-get, hx-post, and hx-delete for CRUD operations
# - Target endpoints on http://127.0.0.1:8000
# - Include a navbar and reload sections dynamically
# - No markdown or code fences
# """
#     html = ask(prompt, model)
#     Path("generated/templates/index.html").write_text(html)
#     print("Saved generated/templates/index.html")
def generate_index_html(models: list, model: str = DEFAULT_MODEL):
    print("Generating index.html...")
    Path("generated/templates").mkdir(parents=True, exist_ok=True)
    prompt = f"""
Write a valid HTML (no markdown fences) Jinja2 template using TailwindCSS and HTMX
for models {models}. Requirements:
- Use the Tailwind CDN for simplicity.
- Each model has a CRUD section with a table and a form.
- Forms use hx-post, hx-delete, and hx-get to interact with the FastAPI endpoints on port 8000.
- Use hx-target="this" and hx-swap="outerHTML" for inline updates.
- Wrap each section in a <div id="{{ model_name|lower }}"> so the target exists.
- Include a <nav> with links to each model section.
- Handle HTMX errors gracefully with hx-on::error to show an alert.
"""
    html = ask(prompt, model)
    Path("generated/templates/index.html").write_text(html)
    print("Saved generated/templates/index.html")
    
def count_code_lines():
    total_lines = 0
    files = ["generated/schema.py", 
             "generated/api.py", 
             "generated/frontend.py",
             "generated/templates/index.html"]
    for file in files:
        try:
            with open(file, "r", encoding="utf-8") as f:
                line_count = sum(1 for _ in f)
            #print(f"{file}: {line_count} lines")
            total_lines += line_count
        except FileNotFoundError:
            print(f"{file}: File not found.")
    return total_lines

# -------------------------------
# Step 8: Main orchestration
# -------------------------------
def main():
    user_prompt = input("Describe your app: ")
    model_choice = input("Which model to use? (default: gpt-4o-mini): ") or "gpt-4o-mini"

    total_tokens = 0
    total_cost = 0.0
    total_start = time.time()

    Path("generated").mkdir(exist_ok=True)
    Path("generated/__init__.py").touch(exist_ok=True)

    # Step 1
    analysis = analyze_prompt(user_prompt, model_choice)
    total_tokens += getattr(analysis, "tokens", 0) if hasattr(analysis, "tokens") else 0

    # Step 2
    generate_schema(analysis, model_choice)
    models = extract_model_names()

    # Step 3
    generate_api()
    generate_frontend(models, model_choice)
    generate_index_html(models, model_choice)

    total_end = time.time()
    total_time = total_end - total_start
    code_lines = count_code_lines()

    print("\n================================================================")
    print("RUN SUMMARY")
    print(f"Model: {model_choice}")
    print(f"Input Prompt: {user_prompt}")
    print(f"Total tokens used: {TOTAL_TOKENS:,}")
    print(f"Estimated total cost: ${TOTAL_COST:.5f}")
    print(f"Total runtime: {total_time:.2f} seconds")
    print(f"Lines of code generated: {code_lines}")
    print("==================================================================\n")
    print("\nExecution Instructions:")
    print("Run backend:  uvicorn generated.api:app --reload --port 8000")
    print("Run frontend: uvicorn generated.frontend:app --reload --port 8001")
    print("Then open http://127.0.0.1:8001 ")



if __name__ == "__main__":
    main()


Describe your app:  an application to track customer feedback
Which model to use? (default: gpt-4o-mini):  anthropic/claude-sonnet-4-5-20250929


Analyzing app idea: an application to track customer feedback
18.13s | 1498 tokens | $0.00449
Schema plan:
 # Database Schema for Customer Feedback Tracking Application

## Entities, Fields, and Relationships

### 1. **Customer**
Stores information about customers who provide feedback.

**Fields:**
- `id` (Integer, Primary Key, Auto-increment)
- `email` (String, Unique, Indexed, Required)
- `first_name` (String, Required)
- `last_name` (String, Required)
- `company` (String, Optional)
- `phone` (String, Optional)
- `created_at` (DateTime, Auto-generated)
- `updated_at` (DateTime, Auto-updated)

### 2. **Product**
Represents products or services that customers provide feedback about.

**Fields:**
- `id` (Integer, Primary Key, Auto-increment)
- `name` (String, Unique, Required)
- `description` (Text, Optional)
- `category` (String, Optional)
- `is_active` (Boolean, Default: True)
- `created_at` (DateTime, Auto-generated)
- `updated_at` (DateTime, Auto-updated)

### 3. **Feedback**
Main e